In [1]:
import os
import json
from datetime import datetime
from bs4 import BeautifulSoup

In [2]:
print(datetime.now().strftime("%Y/%m/%d %H:%M:%S"))

2024/02/16 00:05:06


In [3]:
slozka = "filmy"

In [4]:
def seznam_souboru(slozka):
    seznam = []
    for file in os.listdir(slozka):
        seznam.append(file)
    return(seznam)

In [5]:
seznam_filmu = seznam_souboru(slozka)
seznam_filmu[0:10]

['10144-tabor-padlych-zen.html',
 '10173-eastern-krvavy-hugo.html',
 '102282-o-jecminkovi.html',
 '102314-zimni-vila.html',
 '10242-narkoblues.html',
 '103341-jan-hus-mse-za-tri-mrtve-muze.html',
 '103354-obcansky-prukaz.html',
 '103358-7-dni-hrichu.html',
 '103360-vojta-lavicka-nahoru-a-dolu.html',
 '103369-kuky-se-vraci.html']

## Začínáme scrapovat

### Nejdřív zjistíme, co všechno budeme scrapovat

In [6]:
kolik = None

(Čím menší číslo, tím menší vzorek a tím rychlejší test. Pro zpracování kompletního seznamu nutno změnit proměnou na None.)

In [7]:
def jake_jsou_sekce(slozka):
    sekce = []
    for s in seznam_souboru(slozka)[:kolik]:
        with open(os.path.join(slozka, s), "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f, "html.parser")
            item_details = soup.find_all(class_='db-film-section')
            for item_detail in item_details:
                h2_tags = item_detail.find_all('h2')
                for h2_tag in h2_tags:
                    if h2_tag.get_text() not in sekce:
                        sekce.append(h2_tag.get_text())
    return(sekce)

In [8]:
sekce = jake_jsou_sekce("filmy")

In [9]:
sekce

['Anotace',
 'Obsah',
 'Hrají',
 'Štáb a tvůrci',
 'Hudba',
 'Lokace',
 'Produkční údaje',
 'Technické údaje',
 'Knihovna',
 'Poznámka',
 'Ocenění',
 'Galerie',
 'Komentář',
 'Písně',
 'Revue',
 'Účinkují',
 'Mluví',
 'Film online']

In [10]:
def jake_jsou_detaily(slozka):
    detaily = []
    for s in seznam_souboru(slozka)[:kolik]:
        with open(os.path.join(slozka, s), "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f, "html.parser")
            item_details = soup.find_all(class_='item-details')
            for item_detail in item_details:
                h3_tags = item_detail.find_all('h3')
                for h3_tag in h3_tags:
                    if h3_tag.get_text() not in detaily:
                        detaily.append(h3_tag.get_text())
    return(detaily)

In [11]:
detaily = jake_jsou_detaily(slozka)

In [12]:
def jake_jsou_kredity(slozka):
    kredity = []
    for s in seznam_souboru(slozka)[:kolik]:
        with open(os.path.join(slozka, s), "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f, "html.parser")
            item_details = soup.find_all(class_='db-film-credits-section')
            for item_detail in item_details:
                if "Štáb a tvůrci" in item_detail.text:
                    h3_tags = item_detail.find_all('h3')
                    for h3_tag in h3_tags:
                        if h3_tag.get_text() not in kredity:
                            kredity.append(h3_tag.get_text())
    return(kredity)

In [13]:
kredity = jake_jsou_kredity(slozka)

In [14]:
def jake_jsou_technicke_detaily(slozka):
    technika = []
    for s in seznam_souboru(slozka)[:kolik]:
        with open(os.path.join(slozka, s), "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f, "html.parser")
            item_details = soup.find_all(class_='db-film-credits-section')
            for item_detail in item_details:
                if "Technické údaje" in item_detail.text:
                    h3_tags = item_detail.find_all('h3')
                    for h3_tag in h3_tags:
                        if h3_tag.get_text() not in technika:
                            technika.append(h3_tag.get_text())
    return(technika)

In [ ]:
technika = jake_jsou_technicke_detaily(slozka)

In [ ]:
technika

## Scrapování filmů

In [ ]:
def id_persony(url):
    if "cs/person/" in url:
        id_persony = url.split("/")[-2]
    else:
        id_persony = 0
    return(id_persony)

In [ ]:
def pridej_atributy(slovnik, seznam, nalezene):
    for s in seznam:
        for n in nalezene:
            if s in n:
                n = n.replace(s," ")
                slovnik[s] = n.strip()

In [ ]:
with open(os.path.join("data_fixes", "zeny_rucne.json"), encoding="utf-8") as zeny:
    zeny = json.loads(zeny.read())

In [ ]:
def scrape_film(slozka, soubor):
    
    def jmeno_nejmeno(div):
        global zeny
        try:
            jmeno = id_persony(div.find("h3").find("a")["href"])   
        except:
            jmeno = div.find("h3").text
        if "/ž/" in div.find("h3").text:
            zena = div.find("h3").text.split("/ž/")[0].strip()
            if zena not in zeny:
                zeny.append(zena)
        return(jmeno)
            
    with open(os.path.join(slozka, soubor), "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")
    
    atributy = {}
        
    atributy["Film"] = f"""{soup.find("h1").text.strip()} ({soup.find('meta', {'property': 'og:url'})['content'].split("/")[-2]})"""

    ## Detaily – země původu, copyright…
    
    item_details = soup.find_all(class_='item-details')
    divs_text = []

    for item_detail in item_details:
        divs = item_detail.find_all('div')
        for div in divs:
            obsah = div.text
            if div.find("span"):
                obsah = div.find("h3").text + ", ".join([s.text for s in div.find_all(class_="comma")])
            divs_text.append(obsah)

    pridej_atributy(atributy, detaily, divs_text)
    
    ## Credits
    
    item_details = soup.find_all(class_='db-film-credits-section')
    divs_text = []
    mluvi = []

    for item_detail in item_details:
        
        if "Hrají" in item_detail.find("h2").text:
            divs = item_detail.find_all(class_="credit-item")
            hraji = []
            postavy = []
            
            for div in divs:
                jmeno = jmeno_nejmeno(div) 
                hraji.append(jmeno)
                if div.find("p"):
                    role = div.find("p").text
                    if div.find("p").find("a"):
                        role = role.replace(div.find("p").find("a").text,"")
                        mluvi.append(id_persony(div.find("a")["href"]))
                    if div.find("p").find("span"):
                        role = role.replace(div.find("p").find("span").text,"")
                    postavy.append(role.strip())
            atributy["Hrají"] = hraji
            atributy["Postavy"] = postavy
            
        if "Mluví" in item_detail.find("h2").text:
            divs = item_detail.find_all(class_="credit-item")
            for div in divs:
                jmeno = jmeno_nejmeno(div)
                mluvi.append(jmeno)
            
        if mluvi:
            atributy["Mluví"] = mluvi
            
        if "Komentář" in item_detail.find("h2").text:
            komentar = []
            divs = item_detail.find_all(class_="credit-item")
            for div in divs:
                jmeno = jmeno_nejmeno(div)
                komentar.append(jmeno)
            atributy["Komentář"] = komentar

        if "Účinkují" in item_detail.find("h2").text:
            ucinkuji = []
            divs = item_detail.find_all(class_="credit-item")
            for div in divs:
                try:
                    jmeno = id_persony(div.find("a")["href"])
                except:
                    jmeno = div.find("h3").text 
                ucinkuji.append(jmeno)
            atributy["Účinkují"] = ucinkuji
            
        if "Štáb a tvůrci" in item_detail.find("h2").text:
            divs = item_detail.find_all(class_='credit-item')
            for div in divs:
                jmena_v_odkazech = []
                odkazy = div.find_all("a")
                for o in odkazy:
                    jmena_v_odkazech.append(o.text.strip())
                jmena_vsude = div.find("p").text.strip().split(",")
                jmena_vsude = [j.replace(", ","") for j in jmena_vsude if j]
                jmena_mimo_odkazy = [j.strip() for j in jmena_vsude if j.strip() not in jmena_v_odkazech]
                ids = [id_persony(o["href"]) for o in odkazy]
                if not ids:
                    ids = []
                if not jmena_mimo_odkazy:
                    jmena_mimo_odkazy = []
                divs_text.append(f"""{div.find("h3").text} {(", ".join(ids))}, {(", ").join(jmena_mimo_odkazy)}""")
                            
            pridej_atributy(atributy, kredity, divs_text)
            
        if "Technické údaje" in item_detail.find("h2").text or "Produkční údaje" in item_detail.find("h2").text:
            divs = item_detail.find_all('div')
            for div in divs:
                if div.find("h3"):
                    if div.find("p"):
                        if div.find("h3").text.strip() == "Zvuk":
                            atributy["Zvukový"] = div.find("p").text.strip()
                        else:
                            atributy[div.find("h3").text.strip()] = div.find("p").text.strip()
                        if "Žánr" in div.find("h3").text:
                            zanry = [z.strip() for z in div.find("p").text.split(",")]
                            zanry = ", ".join(zanry)
                            atributy[div.find("h3").text.strip()] = zanry
                        if div.find(class_="comma"):
                                atributy[div.find("h3").text.strip()] = ", ".join([s.text.strip() for s in div.find_all(class_="comma")])
                        if div.find("h3").text.strip() in ["Výrobce","S podporou","Nositelé copyrightu","Distribuce"]:
                            odkazy = div.find_all("a")
                            jmena_v_odkazech = [id_persony(o["href"]) for o in odkazy]
                            atributy[div.find("h3").text.strip()] = jmena_v_odkazech
                        
        if "Písně" in item_detail.find("h2").text:
            pisne = item_detail.find_all('h3')
            pisne = [pisen.text.strip() for pisen in pisne]
            atributy["Písně"] = pisne
            atributy["Hudba, text a zpěv písně"] = [id_persony(a["href"]) for a in item_detail.find_all("a")]
            
        if "Lokace" in item_detail.find("h2").text:
            lokace = item_detail.find('p')
            atributy["Lokace"] = lokace.text.strip()

        if "Ocenění" in item_detail.find("h2").text:
            oceneni = item_detail.find_all(class_="col-md-3")
            oceneni = [cena.text.strip() for cena in oceneni]
            atributy["Ocenění"] = oceneni
            
        if "Hudba" in item_detail.find("h2").text:
            divs = item_detail.find_all('div')
            for div in divs:
                atributy[div.find("h3").text] = [id_persony(a["href"]) for a in div.find_all("a")]
            
    item_details = soup.find_all(class_='film-description')
    
    for item_detail in item_details:
    
        if "Anotace" in item_detail.find("h2").text:
            atributy["Anotace"] = item_detail.find('p').text.strip()
            
        if "Obsah" in item_detail.find("h2").text:
            atributy["Obsah"] = item_detail.find('p').text.strip()
        
        if "Poznámka" in item_detail.find("h2").text:
            atributy["Poznámka"] = item_detail.find('p').text.strip()
        
    return(atributy)

## Test

In [ ]:
scrape_film(slozka, "401929-skleneny-pokoj.html")

In [ ]:
scrape_film(slozka, "395251-manzele-pani-mileny.html")

In [ ]:
zeny

## Konečně DataFrame

In [ ]:
import pandas as pd

In [ ]:
kolik = None

In [ ]:
def frejm(slozka):
    df = pd.DataFrame()
    total = len(seznam_souboru(slozka)[:kolik])
    count = 0
    for s in seznam_souboru(slozka)[:kolik]:
        count = count + 1
        print(f"{count} / {total} / {s}")
        line = pd.DataFrame.from_dict([scrape_film(slozka, s)])
        df = pd.concat([df, line])
    df = df.reset_index(drop = True)
    return(df)

In [ ]:
df = frejm("filmy")
df

In [ ]:
df["Asistent vedoucího produkce"].drop_duplicates().to_list()

In [ ]:
df.columns

## Export

In [ ]:
with open(os.path.join("data_fixes", "zeny.json"), "w+", encoding="utf-8") as zeny_soubor:
    json.dump(zeny, zeny_soubor)

In [ ]:
df.to_json(os.path.join("data", "filmy_raw.json"), orient="records")

In [ ]:
print(datetime.now().strftime("%Y/%m/%d %H:%M:%S"))